In [ ]:
import numpy as np
import torch

from lib.config.project_manager import ProjectManager
from lib.dataset.dataset2D import Dataset2D
from lib.hybridnet.efficienttrack.efficienttrack import EfficientTrack

project = ProjectManager()
project.load('Test_Project')

#project.create_new(
#    name = 'Face_Pretrain',
#    dataset2D_path = '/home/trackingsetup/Documents/Vortex/datasets/WFLW_Set',
#    dataset3D_path = None)

cfg = project.get_cfg()

In [ ]:
training_set = Dataset2D(project.cfg, set='train', mode = 'KeypointDetect')
val_set = Dataset2D(project.cfg, set='val',mode = 'KeypointDetect')

In [ ]:
weights = '/home/trackingsetup/Downloads/EfficientTrack-d2_100.pth'

efficientTrack = EfficientTrack('KeypointDetect', cfg, None)

In [ ]:
efficientTrack.train(training_set, val_set, 1000)

In [ ]:
import lib.hybridnet.modules.efficienttrack.darkpose as darkpose
import cv2
import matplotlib.pyplot as plt

weights = "/home/timo/Desktop/HybridNet/projects/Test_Project/models/KeypointDetect/Run_20211120-160448/EfficientTrack-d0_70.pth"

efficientTrack = EfficientTrack('inference', cfg, weights)

In [ ]:
#j = 0
item = val_set.__getitem__(j)
j += 1
imgs = item[0]
img = ((imgs*cfg.DATASET.STD)+cfg.DATASET.MEAN)*255

imgs = torch.from_numpy(imgs).permute(2, 0, 1).float()
print (imgs.shape)
imgs = imgs.reshape(1,3,256,256)
imgs = imgs.cuda()
outputs = efficientTrack.model(imgs)
preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)

print (maxvals)

colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                  (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]
for i,point in enumerate(preds[0]):
    if (maxvals[0][i]) > 50:
        cv2.circle(img, (int(point[0]*2), int(point[1])*2), 2, (255,100,100), thickness=5)
    else:
        cv2.circle(img, (int(point[0]*2), int(point[1])*2), 2, (100,100,100), thickness=5)

plt.imshow(img/255.)
#hetamap = cv2.resize(outputs[1].clamp(0,255).detach().cpu().numpy()[0][12]/255., (256,256), interpolation=cv2.cv2.INTER_NEAREST)
#plt.imshow(hetamap, alpha=0.6)
plt.imsave( 'test.png',img/255.)
#plt.savefig('test.png')